In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

!pip install kaggle
!pip install -q ultralytics pycocotools easyocr opencv-python pillow matplotlib tqdm scikit-image
!pip install rembg
!pip install onnxruntime onnxruntime-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (10.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126455 files and direc

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_USERNAME'] = 'starsin'
os.environ['KAGGLE_KEY'] = 'ff559a02afd29d67f19bc4001d5a69f7'

api = KaggleApi()
api.authenticate()


# 2. 대회 데이터 다운로드
competition_name = "ai05-level1-project"
api.competition_download_files(competition_name, path="./data")

In [ ]:
import zipfile

zip_path = "./data/ai05-level1-project.zip"
extract_dir = "./data/ai05-level1-project"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

## 기본 세팅 & 경로

In [ ]:
# === Cell 1: 공통 설정 & 유틸 ===
import os, json, math, shutil, random, glob
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Tuple
from PIL import Image
import pandas as pd
import numpy as np

# ---- 경로 설정 (환경에 맞게 조정) ----
DATA_ROOT      = "./data/ai05-level1-project"
TRAIN_IMG_DIR  = f"{DATA_ROOT}/train_images"
TEST_IMG_DIR   = f"{DATA_ROOT}/test_images"
ANN_DIR        = f"{DATA_ROOT}/train_annotations"   # *_json 폴더 구조
WORK_DIR       = "./work_yolov8"

ALL_JSON       = f"{WORK_DIR}/all.json"             # 병합 COCO
CLEAN_JSON     = f"{WORK_DIR}/all_clean.json"       # 클린 COCO
KFOLD_DIR      = f"{WORK_DIR}/kfold"                # 폴드 COCO 저장
YOLO_ROOT      = f"{WORK_DIR}/yolo_splits"          # 폴드별 YOLO 데이터셋
PRED_DIR       = f"{WORK_DIR}/predictions"

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(KFOLD_DIR, exist_ok=True)
os.makedirs(YOLO_ROOT, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# ---- 파라미터(필요시 조정) ----
SEED                    = 42
random.seed(SEED); np.random.seed(SEED)
N_FOLDS                 = 5
K_PER_CLASS_IN_VAL      = 1    # 각 클래스 최소 커버
MIN_BBOX_WH             = 2    # bbox 최소 너비/높이 (px)
MAX_IMAGE_COVER_RATIO   = 0.95 # 박스가 이미지 95% 이상이면 제거
DEDUP_ROUND_DIGITS      = 1    # 중복판정 소수점 자리
IMG_EXTS                = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

def image_size_from_disk(file_name: str) -> Tuple[int,int]:
    p = os.path.join(TRAIN_IMG_DIR, file_name)
    if not os.path.isfile(p): return (0,0)
    try:
        with Image.open(p) as im:
            return im.size
    except:
        return (0,0)

def is_image_missing(file_name: str) -> bool:
    return not os.path.isfile(os.path.join(TRAIN_IMG_DIR, file_name))

def clip_bbox(x,y,w,h,W,H):
    if any(v is None for v in [x,y,w,h]): return None
    if w <= 0 or h <= 0: return None
    x = max(0, min(float(x), max(0, W-1)))
    y = max(0, min(float(y), max(0, H-1)))
    w = max(float(MIN_BBOX_WH), min(float(w), max(1.0, W - x)))
    h = max(float(MIN_BBOX_WH), min(float(h), max(1.0, H - y)))
    if (w*h) > (MAX_IMAGE_COVER_RATIO * W * H):
        return None
    return [float(x), float(y), float(w), float(h)]

def ann_key(img_id, cat_id, bbox, digits=DEDUP_ROUND_DIGITS):
    x,y,w,h = [round(float(v), digits) for v in bbox]
    return f"{int(img_id)}_{int(cat_id)}_{x}_{y}_{w}_{h}"

def load_coco(path: str) -> Dict:
    with open(path, "r", encoding="utf-8") as f:
        d = json.load(f)
    for k in ["images","annotations","categories"]:
        assert k in d, f"COCO key missing: {k}"
    return d

def save_coco(d: Dict, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(d, f, ensure_ascii=False, indent=2)

def ensure_min_class_coverage(val_img_ids: set, img_to_cats: Dict[int,set], target_min=1) -> set:
    class_to_imgs_val = defaultdict(set)
    for iid in val_img_ids:
        for c in img_to_cats.get(iid, set()):
            class_to_imgs_val[c].add(iid)

    need = {}
    all_classes = set()
    for s in img_to_cats.values(): all_classes |= s
    for c in all_classes:
        cur = len(class_to_imgs_val.get(c, set()))
        if cur < target_min:
            need[c] = target_min - cur

    if not need: return val_img_ids

    all_img_ids = set(img_to_cats.keys())
    train_pool = list(all_img_ids - set(val_img_ids))
    random.shuffle(train_pool)

    for c, deficit in need.items():
        if deficit <= 0: continue
        for iid in train_pool:
            if iid in val_img_ids: continue
            if c in img_to_cats.get(iid, set()):
                val_img_ids.add(iid)
                deficit -= 1
                if deficit <= 0: break

    return val_img_ids

## 누수 stem 합치기(중복 제거 목록 생성)

In [ ]:
# === Cell 2-1: 누수 stem 수집 + 남은 test 이미지 확인 ===
import pandas as pd

WORK_DATA_DIR = "./data"  # 누수 CSV가 저장된 디렉토리
CSV_NAME   = os.path.join(WORK_DATA_DIR, "leak_name_stem_overlap.csv")      # cols: test_path, stem
CSV_MD5    = os.path.join(WORK_DATA_DIR, "leak_exact_hash_md5.csv")         # cols: test_path, train_path, md5
CSV_AHASH  = os.path.join(WORK_DATA_DIR, "leak_perceptual_ahash.csv")       # cols: test_path, train_path, hamming
CSV_SSIM   = os.path.join(WORK_DATA_DIR, "leak_ssim_strict097.csv")         # cols: test_path, train_path, ssim

def _safe_read_csv(p):
    return pd.read_csv(p) if os.path.isfile(p) else pd.DataFrame()

def _stem(p):
    return Path(str(p)).stem

# 1) name stem 중복
# df_name = _safe_read_csv(CSV_NAME)
# stems_name = set(df_name["stem"].astype(str)) if "stem" in df_name.columns else set()

# 2) MD5 / ahash / ssim → test_path에서 stem 추출
stems_md5 = set(_stem(p) for p in _safe_read_csv(CSV_MD5).get("test_path", []))
# stems_ah  = set(_stem(p) for p in _safe_read_csv(CSV_AHASH).get("test_path", []))
# stems_ss  = set(_stem(p) for p in _safe_read_csv(CSV_SSIM).get("test_path", []))

LEAK_STEMS = stems_md5 # stems_name | stems_md5 | stems_ah | stems_ss
print(f"🔎 leak stems collected: {len(LEAK_STEMS)}개")

# test 폴더의 전체 stem
test_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(IMG_EXTS)]
test_stems = {Path(f).stem for f in test_files}

non_leak_stems = sorted(test_stems - LEAK_STEMS)    # 정상
leak_stems_in_test = sorted(test_stems & LEAK_STEMS) # 누수

print(f"\n📊 test 폴더 분석")
print(f" - 전체: {len(test_stems)}")
print(f" - 누수: {len(leak_stems_in_test)}")
print(f" - 정상: {len(non_leak_stems)}")

# (옵션) CSV 저장
pd.DataFrame({"stem": non_leak_stems}).to_csv(os.path.join(WORK_DIR, "test_non_leak_stems.csv"), index=False, encoding="utf-8-sig")
pd.DataFrame({"stem": leak_stems_in_test}).to_csv(os.path.join(WORK_DIR, "test_leak_stems.csv"),     index=False, encoding="utf-8-sig")
print("✅ 목록 CSV 저장 완료")

🔎 leak stems collected: 838개

📊 test 폴더 분석
 - 전체: 843
 - 누수: 838
 - 정상: 5
✅ 목록 CSV 저장 완료


In [ ]:
# === Cell 2-2: test_images 폴더에서 누수 stem 파일 제거 ===
DRY_RUN = True   # 실제 삭제하려면 False로 설정
def _list_images(d):
    return [os.path.join(d, f) for f in os.listdir(d) if f.lower().endswith(IMG_EXTS)]

test_files_full = _list_images(TEST_IMG_DIR)
to_delete = [p for p in test_files_full if Path(p).stem in LEAK_STEMS]

print(f"🧹 삭제 대상(test_images): {len(to_delete)}개")
for p in to_delete[:10]:
    print(" -", os.path.relpath(p, TEST_IMG_DIR))

if not DRY_RUN:
    removed = 0
    for p in to_delete:
        try:
            os.remove(p)
            removed += 1
        except Exception as e:
            print("skip(remove err):", p, e)
    print(f"✅ removed {removed} files from test_images")
else:
    print("ℹ️ DRY_RUN=True → 실제 삭제는 수행하지 않음")

🧹 삭제 대상(test_images): 841개
 - 287.png
 - 496.png
 - 560.png
 - 822.png
 - 277.png
 - 1199.png
 - 24.png
 - 435.png
 - 1492.png
 - 180.png
ℹ️ DRY_RUN=True → 실제 삭제는 수행하지 않음


In [2]:
# === Cell 2-2 (final): test 누수 삭제 + non-leak 전부 train으로 이동 (DRY_RUN이어도 실행) ===
import os, shutil, csv
from pathlib import Path
import pandas as pd

# 이미 앞 셀들에서 정의했다면 그대로 사용, 없으면 기본값 세팅
WORK_DIR      = globals().get("WORK_DIR", os.path.abspath("./data"))
DATA_ROOT     = globals().get("DATA_ROOT", os.path.abspath("./data/ai05-level1-project"))
TRAIN_IMG_DIR = globals().get("TRAIN_IMG_DIR", os.path.join(DATA_ROOT, "train_images"))
TEST_IMG_DIR  = globals().get("TEST_IMG_DIR",  os.path.join(DATA_ROOT, "test_images"))
IMG_EXTS      = globals().get("IMG_EXTS", (".jpg",".jpeg",".png",".bmp",".webp"))

# 앞 셀에서 만든 누수 STEM 집합
LEAK_STEMS = globals().get("LEAK_STEMS", set())
if not isinstance(LEAK_STEMS, set):
    LEAK_STEMS = set(LEAK_STEMS)

# 사용자가 요청: DRY_RUN이어도 실제 실행
DRY_RUN = True   # <- 값이 무엇이든 '실행'하도록 동작 (표시용 플래그로만 사용)

os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(TEST_IMG_DIR, exist_ok=True)
os.makedirs(WORK_DIR, exist_ok=True)

def list_images(d):
    return [os.path.join(d, f) for f in os.listdir(d) if f.lower().endswith(IMG_EXTS)]

def unique_dst_path(dst_dir: str, filename: str):
    """
    train에 동명 파일이 있으면 name__t2tr-001.jpg 식으로 안전하게 리네이밍
    """
    name = Path(filename).name
    base = Path(name).stem
    ext  = Path(name).suffix
    candidate = os.path.join(dst_dir, name)
    if not os.path.exists(candidate):
        return candidate
    # 충돌 시 번호 붙이기
    i = 1
    while True:
        cand = os.path.join(dst_dir, f"{base}__t2tr-{i:03d}{ext}")
        if not os.path.exists(cand):
            return cand
        i += 1

# 1) test 이미지 스캔
test_files = list_images(TEST_IMG_DIR)
test_stems = {Path(p).stem for p in test_files}

# 2) 삭제 대상(누수) / 이동 대상(정상) 분리
delete_targets = [p for p in test_files if Path(p).stem in LEAK_STEMS]
move_targets   = [p for p in test_files if Path(p).stem not in LEAK_STEMS]

print(f"🧹 삭제 대상(누수) in test_images: {len(delete_targets)}개")
print(f"🚚 이동 대상(정상) test → train: {len(move_targets)}개")

# 3) 로그 준비
deleted_csv = os.path.join(WORK_DIR, "deleted_in_test.csv")
moved_csv   = os.path.join(WORK_DIR, "moved_from_test_to_train.csv")
deleted_rows, moved_rows = [], []

# 4) 누수 파일 실제 삭제 (DRY_RUN 이라도 실행)
removed = 0
for src in delete_targets:
    try:
        os.remove(src)
        removed += 1
        deleted_rows.append({"path": src, "action": "delete_in_test"})
    except Exception as e:
        deleted_rows.append({"path": src, "action": f"delete_failed: {e}"})
print(f"✅ test 누수 삭제 완료: {removed}/{len(delete_targets)}")

# 5) 정상 파일은 전부 train으로 '이동'(DRY_RUN 이라도 실행)
moved = 0
for src in move_targets:
    try:
        dst = unique_dst_path(TRAIN_IMG_DIR, Path(src).name)
        shutil.move(src, dst)   # 실제 move
        moved += 1
        moved_rows.append({"src": src, "dst": dst, "action": "move_test_to_train"})
    except Exception as e:
        moved_rows.append({"src": src, "dst": "", "action": f"move_failed: {e}"})
print(f"✅ test→train 이동 완료: {moved}/{len(move_targets)}")

# 6) 로그 저장
pd.DataFrame(deleted_rows).to_csv(deleted_csv, index=False, encoding="utf-8-sig")
pd.DataFrame(moved_rows).to_csv(moved_csv,   index=False, encoding="utf-8-sig")

print("\n📁 로그 저장")
print(" - 삭제 로그:", deleted_csv)
print(" - 이동 로그:", moved_csv)

# 7) 요약 프린트 (상위 5개 미리보기)
print("\n🧾 삭제 미리보기 (top 5):")
for r in deleted_rows[:5]:
    print("  ", r)

print("\n🧾 이동 미리보기 (top 5):")
for r in moved_rows[:5]:
    print("  ", r)

🧹 삭제 대상(누수) in test_images: 0개
🚚 이동 대상(정상) test → train: 0개
✅ test 누수 삭제 완료: 0/0
✅ test→train 이동 완료: 0/0

📁 로그 저장
 - 삭제 로그: /content/data/deleted_in_test.csv
 - 이동 로그: /content/data/moved_from_test_to_train.csv

🧾 삭제 미리보기 (top 5):

🧾 이동 미리보기 (top 5):


## COCO 로드 & 기본 유틸

In [ ]:
# === Cell: 어노테이션 및 카테고리 병합 ===
import os, json
from pathlib import Path
from collections import defaultdict
from PIL import Image

# === 설정 ===
ANN_DIR = "./data/ai05-level1-project/train_annotations"              # 폴더별 *_json 구조가 있는 곳
TRAIN_IMG_DIR = "./data/ai05-level1-project/train_images"
ALL_JSON = "./work_yolov8/all.json"

os.makedirs(os.path.dirname(ALL_JSON), exist_ok=True)

# === 유틸 ===
def file_exists_in_train_images(fn):
    return os.path.isfile(os.path.join(TRAIN_IMG_DIR, fn))

def norm_name(fn: str):
    return Path(fn).name.strip().replace("\\", "/")

def get_img_size_from_disk(fn):
    p = os.path.join(TRAIN_IMG_DIR, fn)
    if not os.path.isfile(p):
        return (0, 0)
    try:
        with Image.open(p) as im:
            return im.size
    except:
        return (0, 0)

def bbox_valid_clip(x, y, w, h, W, H):
    """잘못된 bbox 제거 + 이미지 범위 내로 클램핑"""
    if any(v is None for v in [x, y, w, h]): return None
    if w <= 0 or h <= 0: return None
    x = max(0, min(x, W-1))
    y = max(0, min(y, H-1))
    w = max(1, min(w, W-x))
    h = max(1, min(h, H-y))
    return [float(x), float(y), float(w), float(h)]

def hash_cat(c):
    """카테고리 중복 판단용 간단한 해시"""
    return f"{c.get('name','').strip().lower()}|{c.get('supercategory','').strip().lower()}"

def dedup_key(img_id, cat_id, bbox):
    """annotation 중복 방지 키"""
    return f"{img_id}_{cat_id}_{round(bbox[0],1)}_{round(bbox[1],1)}_{round(bbox[2],1)}_{round(bbox[3],1)}"

# === 메인 병합 ===
merged = {"images": [], "annotations": [], "categories": []}
cat_by_hash, cat_id_map = {}, {}
file_to_new_img_id, image_meta = {}, {}
seen_ann_keys = set()

new_img_id, ann_id, new_cat_id = 1, 1, 1
cnt = defaultdict(int)

for folder in sorted(os.listdir(ANN_DIR)):
    if not folder.endswith("_json"):
        continue
    cnt["folders_scanned"] += 1
    for dp, _, files in os.walk(os.path.join(ANN_DIR, folder)):
        for f in files:
            if not f.lower().endswith(".json"):
                continue
            cnt["json_scanned"] += 1
            jp = os.path.join(dp, f)
            try:
                j = json.load(open(jp, "r", encoding="utf-8"))
            except Exception:
                continue

            # === 1) 카테고리 병합 ===
            for c in j.get("categories", []):
                h = hash_cat(c)
                cid = int(c["id"])
                if h not in cat_by_hash:
                    cat_by_hash[h] = {"id": cid, "name": c.get("name", ""), "supercategory": c.get("supercategory", "pill")}
                cat_id_map[c["id"]] = cat_by_hash[h]["id"]

            # === 2) 이미지 및 어노테이션 병합 ===
            for im in j.get("images", []):
                file_name = norm_name(im.get("file_name", ""))
                if not file_exists_in_train_images(file_name):
                    continue
                if file_name not in file_to_new_img_id:
                    W, H = get_img_size_from_disk(file_name)
                    if W == 0 or H == 0:
                        continue
                    file_to_new_img_id[file_name] = new_img_id
                    image_meta[new_img_id] = {
                        "id": new_img_id,
                        "file_name": file_name,
                        "width": W,
                        "height": H,
                    }
                    new_img_id += 1

            for a in j.get("annotations", []):
                src_im = next((im for im in j.get("images", []) if im["id"] == a.get("image_id")), None)
                if not src_im:
                    continue
                file_name = norm_name(src_im.get("file_name", ""))
                if file_name not in file_to_new_img_id:
                    continue
                W, H = get_img_size_from_disk(file_name)
                fixed = bbox_valid_clip(*a["bbox"], W, H)
                if not fixed:
                    continue
                new_imgid = file_to_new_img_id[file_name]
                new_catid = cat_id_map.get(a["category_id"], 1)

                k = dedup_key(new_imgid, new_catid, fixed)
                if k in seen_ann_keys:
                    continue
                seen_ann_keys.add(k)

                merged["annotations"].append({
                    "id": ann_id,
                    "image_id": new_imgid,
                    "category_id": new_catid,
                    "bbox": fixed,
                    "area": float(fixed[2] * fixed[3]),
                    "iscrowd": int(a.get("iscrowd", 0))
                })
                ann_id += 1

merged["images"] = [image_meta[i] for i in sorted(image_meta)]
merged["categories"] = [cat_by_hash[h] for h in sorted(cat_by_hash)]

json.dump(merged, open(ALL_JSON, "w"), indent=2, ensure_ascii=False)
print(f"✅ 병합 완료: {len(merged['images'])} imgs, {len(merged['annotations'])} anns, {len(merged['categories'])} cats")

✅ 병합 완료: 1489 imgs, 4526 anns, 73 cats


## 결측치/이상치 클린업(특히 bbox)

In [ ]:
# === Cell 4 (최종 수정본): train_path 기반 누수 이미지 제거 + 클린업 + 중복 제거 ===
import pandas as pd
from pathlib import Path
import os, json

# =============================
# 1️⃣ 누수 stem 및 train_path 기반 수집
# =============================
LEAK_STEMS = set()
LEAK_TRAIN_FILES = set()

for leak_csv in [
    # "./data/test_leak_stems.csv",          # test 누수 목록
    # "./data/leak_name_stem_overlap.csv",
    "./data/leak_exact_hash_md5.csv",
    # "./data/leak_perceptual_ahash.csv",
    # "./data/leak_ssim_strict097.csv",
]:
    if os.path.isfile(leak_csv):
        try:
            df = pd.read_csv(leak_csv)
            # ✅ test_path 기준 누수 (test 쪽)
            if "stem" in df.columns:
                LEAK_STEMS |= set(df["stem"].astype(str))
            elif "test_path" in df.columns:
                LEAK_STEMS |= {Path(p).stem for p in df["test_path"].astype(str)}
            # ✅ train_path 기준 누수 (train 쪽)
            if "train_path" in df.columns:
                LEAK_TRAIN_FILES |= {Path(p).name for p in df["train_path"].astype(str)}
        except Exception as e:
            print("⚠️ CSV 읽기 오류:", leak_csv, e)

print(f"🚫 누수 stem {len(LEAK_STEMS)}개, train_path 파일 {len(LEAK_TRAIN_FILES)}개 감지")

# =============================
# 2️⃣ COCO 로드
# =============================
coco = load_coco(ALL_JSON)

valid_images = []
file2imgid = {}
new_img_id = 1

# =============================
# 3️⃣ 이미지 필터링 (누수 제거 + 존재 확인)
# =============================
for im in coco["images"]:
    fname = Path(im["file_name"]).name
    stem = Path(fname).stem

    # ① stem 기준 누수 제거
    if stem in LEAK_STEMS:
        continue

    # ② train_path 기준 누수 제거
    if fname in LEAK_TRAIN_FILES:
        continue

    # ③ 파일 존재 확인
    if is_image_missing(fname):
        continue

    W,H = image_size_from_disk(fname)
    if W <= 0 or H <= 0:
        continue

    file2imgid[fname] = new_img_id
    valid_images.append({
        "id": new_img_id,
        "file_name": fname,
        "width": W,
        "height": H
    })
    new_img_id += 1

print(f"📉 필터링 후 이미지 수: {len(valid_images)} (원본 {len(coco['images'])})")

# =============================
# 4️⃣ old_id → new_id 매핑
# =============================
orig_file_by_id = {im["id"]: Path(im["file_name"]).name for im in coco["images"]}
old_to_new = {}
for im in valid_images:
    for oid, fn in orig_file_by_id.items():
        if fn == im["file_name"]:
            old_to_new[oid] = im["id"]

# =============================
# 5️⃣ 어노테이션 정리
# =============================
new_anns, seen = [], set()
for a in coco["annotations"]:
    oid = a.get("image_id")
    if oid not in old_to_new:
        continue
    nid = old_to_new[oid]
    im = next((x for x in valid_images if x["id"] == nid), None)
    if not im:
        continue
    W,H = im["width"], im["height"]
    bb = a.get("bbox")
    if not bb or len(bb) != 4:
        continue
    fixed = clip_bbox(*bb, W, H)
    if not fixed:
        continue
    k = ann_key(nid, a["category_id"], fixed)
    if k in seen:
        continue
    seen.add(k)
    new_anns.append({
        "id": len(new_anns)+1,
        "image_id": nid,
        "category_id": int(a["category_id"]),
        "bbox": fixed,
        "area": float(fixed[2]*fixed[3]),
        "iscrowd": int(a.get("iscrowd", 0))
    })

# =============================
# 6️⃣ 정리된 COCO 저장
# =============================
cleaned = {
    "images": valid_images,
    "annotations": new_anns,
    "categories": coco["categories"]
}
save_coco(cleaned, CLEAN_JSON)

print(f"✅ cleaned 완료 → imgs={len(valid_images)}, anns={len(new_anns)}, cats={len(cleaned['categories'])}")
print("   (train_path + test_path 기반 누수 제거 적용 완료)")

🚫 누수 stem 838개, train_path 파일 838개 감지
📉 필터링 후 이미지 수: 651 (원본 1489)
✅ cleaned 완료 → imgs=651, anns=2435, cats=73
   (train_path + test_path 기반 누수 제거 적용 완료)


## 중복/누수 이미지 제외 (stem 기준)

In [ ]:
# === Cell 4.5: Test Holdout 생성 + 이미지 준비 + K-Fold용 JSON 분리 ===
import os, json, shutil, glob, numpy as np
from pathlib import Path
from collections import defaultdict, Counter
from sklearn.model_selection import StratifiedShuffleSplit

# ── 입력(앞 셀들과 동일 네이밍 가정) ───────────────────────────────────────
# CLEAN_JSON : Cell 4에서 저장한 클린 COCO 경로 (ex: ./work_yolov8/all_clean.json)
# TRAIN_IMG_DIR : 원본 이미지 디렉터리 (ex: ./data/ai05-level1-project/train_images)
assert os.path.isfile(CLEAN_JSON), f"CLEAN_JSON not found: {CLEAN_JSON}"
assert os.path.isdir(TRAIN_IMG_DIR), f"TRAIN_IMG_DIR not found: {TRAIN_IMG_DIR}"

WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

# ── 파라미터 ───────────────────────────────────────────────────────────────
HOLDOUT_RATIO = 0.10        # 전체 이미지 중 test로 홀드아웃할 비율
SEED          = 42
K_PER_CLASS_IN_TEST = 1     # test에 각 클래스 최소 1장 이상 담기 시도
COPY_INSTEAD_OF_SYMLINK = False  # 권한 문제 있으면 True로

# ── 유틸 ───────────────────────────────────────────────────────────────────
def load_coco(p):
    with open(p, "r", encoding="utf-8") as f:
        d = json.load(f)
    for k in ["images","annotations","categories"]:
        if k not in d: raise ValueError(f"bad COCO (missing {k}): {p}")
    return d

def save_coco(obj, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def primary_label_per_image(coco_dict):
    img_to_cats = defaultdict(list)
    for a in coco_dict["annotations"]:
        img_to_cats[a["image_id"]].append(int(a["category_id"]))
    y = []
    ids = [im["id"] for im in coco_dict["images"]]
    for iid in ids:
        cats = img_to_cats.get(iid, [])
        if not cats:
            y.append(-1)  # 어노 없는 이미지
        else:
            cnt = Counter(cats)
            y.append(sorted(cnt.items(), key=lambda x: (-x[1], x[0]))[0][0])
    return ids, np.array(y)

def ensure_min_class_coverage(selected_ids: set, img_to_catset: dict, target_min=1):
    """validation/test에 각 클래스 최소 target_min 보장하도록 약하게 조정"""
    if target_min <= 0: return selected_ids
    sel = set(selected_ids)
    # 부족 클래스 찾기
    cat_counts = Counter(c for iid in sel for c in img_to_catset.get(iid, []))
    need = {c for c, n in cat_counts.items() if n < target_min}
    if not need:
        # 아예 안 들어간 클래스도 체크
        all_cats = set(c for s in img_to_catset.values() for c in s)
        need = {c for c in all_cats if c not in cat_counts}
    if not need:
        return sel
    # 부족 클래스 보충: sel에 없는 이미지 중 해당 클래스 포함하는 것 추가
    pool = [iid for iid in img_to_catset.keys() if iid not in sel]
    for c in need:
        for iid in pool:
            if c in img_to_catset[iid]:
                sel.add(iid)
                break
    return sel

# ── 1) CLEAN_JSON 로드 & 라벨 벡터 ─────────────────────────────────────────
clean = load_coco(CLEAN_JSON)
img_ids_all, y_all = primary_label_per_image(clean)

# 어노 없는 이미지 제외(−1 제거)
mask = y_all >= 0
img_ids_labeled = np.array(img_ids_all)[mask]
y_labeled       = y_all[mask]

# 각 이미지의 카테고리 집합 (coverage용)
img_to_catset = defaultdict(set)
for a in clean["annotations"]:
    img_to_catset[a["image_id"]].add(int(a["category_id"]))

# ── 2) Stratified Shuffle로 test holdout 추출 ─────────────────────────────
sss = StratifiedShuffleSplit(
    n_splits=1, test_size=HOLDOUT_RATIO, random_state=SEED
)
tr_idx, te_idx = next(sss.split(img_ids_labeled, y_labeled))
train_pool_ids = set(img_ids_labeled[tr_idx])
test_ids       = set(img_ids_labeled[te_idx])

# 최소 클래스 커버 보장(테스트 세트)
test_ids = ensure_min_class_coverage(test_ids, img_to_catset, target_min=K_PER_CLASS_IN_TEST)
# 나머지는 학습 풀로
train_pool_ids = set(img_ids_labeled) - set(test_ids)

print(f"🧪 Holdout split → train_pool:{len(train_pool_ids)} | test:{len(test_ids)} "
      f"({HOLDOUT_RATIO*100:.1f}% target, coverage≥{K_PER_CLASS_IN_TEST})")

# ── 3) test_holdout.json / train_for_kfold.json 생성 ───────────────────────
def dump_subset(src, keep_img_ids: set, out_json: str):
    keep = set(keep_img_ids)
    images = [im for im in src["images"] if im["id"] in keep]
    ann    = [a for a in src["annotations"] if a["image_id"] in keep]
    out = {"images": images, "annotations": ann, "categories": src["categories"]}
    save_coco(out, out_json)

TEST_JSON = os.path.join(WORK_DIR, "test_holdout.json")
TRAIN_FOR_KFOLD_JSON = os.path.join(WORK_DIR, "train_for_kfold.json")
dump_subset(clean, test_ids, TEST_JSON)
dump_subset(clean, train_pool_ids, TRAIN_FOR_KFOLD_JSON)
print("✅ saved:", TEST_JSON, "|", TRAIN_FOR_KFOLD_JSON)

# ── 4) 테스트용 이미지 디렉터리 구성(심볼릭 링크 또는 복사) ────────────────
TEST_DIR_LINK = os.path.join(WORK_DIR, "_holdout_imgs")
os.makedirs(TEST_DIR_LINK, exist_ok=True)

# 폴더 비우기
for p in Path(TEST_DIR_LINK).glob("*"):
    try:
        if p.is_symlink() or p.is_file(): p.unlink()
        elif p.is_dir(): shutil.rmtree(p)
    except Exception: pass

# 링크/복사 수행
name_set = {Path(im["file_name"]).name for im in load_coco(TEST_JSON)["images"]}
linked = 0
for name in sorted(name_set):
    src = Path(TRAIN_IMG_DIR) / name
    if not src.exists():
        continue
    dst = Path(TEST_DIR_LINK) / name
    try:
        if COPY_INSTEAD_OF_SYMLINK:
            shutil.copy2(src, dst)
        else:
            os.symlink(src, dst)
        linked += 1
    except FileExistsError:
        pass
    except OSError:
        # 심볼릭 링크 실패 시 복사 fallback
        shutil.copy2(src, dst)
        linked += 1

print(f"🧷 test images prepared: {linked} files → {TEST_DIR_LINK}")

# ── 5) 아래 셀들이 참조할 전역 경로 지정 ───────────────────────────────────
#  - Cell 5에서는 CLEAN_JSON 대신 CLEAN_JSON_FOR_KFOLD를 읽도록 변경
CLEAN_JSON_FOR_KFOLD = TRAIN_FOR_KFOLD_JSON
TEST_DIR = TEST_DIR_LINK

# 요약 출력
print("\n[SUMMARY]")
print(" - CLEAN_JSON_FOR_KFOLD:", CLEAN_JSON_FOR_KFOLD)
print(" - TEST_JSON           :", TEST_JSON)
print(" - TEST_DIR            :", TEST_DIR)

[eligible stratify] classes>1: 6, samples: 648 | singletons: 3
🧪 Holdout split (safe) → train_pool:586 | test:65


In [ ]:
# === Cell 4.6: Holdout JSON 저장 + 테스트 이미지 디렉터리 준비 ===
import os, json, shutil
from pathlib import Path

# 앞 셀에서 이미 존재해야 함
assert 'train_pool_ids' in globals() and 'test_ids' in globals(), "train_pool_ids / test_ids가 필요합니다."
assert 'CLEAN_JSON' in globals(), "CLEAN_JSON 경로를 찾을 수 없습니다."
assert os.path.isfile(CLEAN_JSON), f"CLEAN_JSON not found: {CLEAN_JSON}"
assert os.path.isdir(TRAIN_IMG_DIR), f"TRAIN_IMG_DIR not found: {TRAIN_IMG_DIR}"

WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

TEST_JSON            = os.path.join(WORK_DIR, "test_holdout.json")
TRAIN_FOR_KFOLD_JSON = os.path.join(WORK_DIR, "train_for_kfold.json")
TEST_DIR_LINK        = os.path.join(WORK_DIR, "_holdout_imgs")  # 이후 예측 등에 사용할 테스트 이미지 폴더

def _load_coco(p):
    with open(p, "r", encoding="utf-8") as f:
        d = json.load(f)
    for k in ["images","annotations","categories"]:
        if k not in d: raise ValueError(f"bad COCO (missing {k}): {p}")
    return d

def _save_coco(obj, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def _dump_subset(src, keep_img_ids: set, out_json: str):
    keep = set(keep_img_ids)
    images = [im for im in src["images"] if im["id"] in keep]
    ann    = [a for a in src["annotations"] if a["image_id"] in keep]
    out = {"images": images, "annotations": ann, "categories": src["categories"]}
    _save_coco(out, out_json)
    return len(images), len(ann)

# 1) CLEAN_JSON에서 서브셋 저장
clean = _load_coco(CLEAN_JSON)
n_te_img, n_te_ann = _dump_subset(clean, test_ids, TEST_JSON)
n_tr_img, n_tr_ann = _dump_subset(clean, train_pool_ids, TRAIN_FOR_KFOLD_JSON)

print(f"✅ saved: {TEST_JSON}        | images={n_te_img}, anns={n_te_ann}")
print(f"✅ saved: {TRAIN_FOR_KFOLD_JSON} | images={n_tr_img}, anns={n_tr_ann}")

# 2) 테스트 이미지 폴더 준비(링크/복사)
#    - 권한/환경에 따라 심볼릭 링크가 안되면 복사로 fallback
for p in Path(TEST_DIR_LINK).glob("*"):
    try:
        if p.is_symlink() or p.is_file(): p.unlink()
        elif p.is_dir(): shutil.rmtree(p)
    except Exception: pass
os.makedirs(TEST_DIR_LINK, exist_ok=True)

name_set = {Path(im["file_name"]).name for im in _load_coco(TEST_JSON)["images"]}
linked = 0
for name in sorted(name_set):
    src = Path(TRAIN_IMG_DIR) / name
    if not src.exists():
        continue
    dst = Path(TEST_DIR_LINK) / name
    try:
        os.symlink(src, dst)          # 심볼릭 링크 시도
        linked += 1
    except Exception:
        shutil.copy2(src, dst)        # 실패 시 복사
        linked += 1

print(f"🧷 test images prepared: {linked} files → {TEST_DIR_LINK}")

# 3) 다음 셀들이 참조할 전역 변수 지정
CLEAN_JSON_FOR_KFOLD = TRAIN_FOR_KFOLD_JSON  # K-Fold는 홀드아웃 제외 학습 풀에서만 수행
TEST_DIR = TEST_DIR_LINK

print("\n[SUMMARY]")
print(" - CLEAN_JSON_FOR_KFOLD:", CLEAN_JSON_FOR_KFOLD)
print(" - TEST_JSON           :", TEST_JSON)
print(" - TEST_DIR            :", TEST_DIR)

✅ saved: /content/work_yolov8/test_holdout.json        | images=65, anns=243
✅ saved: /content/work_yolov8/train_for_kfold.json | images=586, anns=2192
🧷 test images prepared: 65 files → /content/work_yolov8/_holdout_imgs

[SUMMARY]
 - CLEAN_JSON_FOR_KFOLD: /content/work_yolov8/train_for_kfold.json
 - TEST_JSON           : /content/work_yolov8/test_holdout.json
 - TEST_DIR            : /content/work_yolov8/_holdout_imgs


## Stratified K-Fold (멀티라벨 근사)

In [ ]:
# === Cell 5: Stratified K-Fold ===
from sklearn.model_selection import StratifiedKFold

d = load_coco(CLEAN_JSON_FOR_KFOLD)

# 이미지 주라벨(가장 많이 등장하는 category_id)
img_to_cats = defaultdict(list)
for a in d["annotations"]:
    img_to_cats[a["image_id"]].append(int(a["category_id"]))

img_ids = [im["id"] for im in d["images"]]
y_primary = []
for iid in img_ids:
    cats = img_to_cats.get(iid, [])
    if not cats:
        y_primary.append(-1)  # 어노 없는 경우
        continue
    cnt = Counter(cats)
    primary = sorted(cnt.items(), key=lambda x: (-x[1], x[0]))[0][0]
    y_primary.append(primary)

img_ids_np, y_np = np.array(img_ids), np.array(y_primary)
mask = y_np >= 0
img_ids_filt, y_filt = img_ids_np[mask], y_np[mask]

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
fold_indices = []
for fold_id, (tr_idx, va_idx) in enumerate(skf.split(img_ids_filt, y_filt), 1):
    tr_ids = set(img_ids_filt[tr_idx])
    va_ids = set(img_ids_filt[va_idx])
    img_to_catset = {iid: set(img_to_cats.get(iid, [])) for iid in img_ids_filt}
    va_ids = ensure_min_class_coverage(va_ids, img_to_catset, target_min=K_PER_CLASS_IN_VAL)
    tr_ids = set(img_ids_filt) - set(va_ids)
    fold_indices.append((fold_id, tr_ids, va_ids))

print("✅ folds:", [(f, len(tr), len(va)) for f,tr,va in fold_indices])

✅ folds: [(1, 466, 120), (2, 468, 118), (3, 464, 122), (4, 465, 121), (5, 466, 120)]


## Fold 저장

In [ ]:
# === Cell 6: 폴드별 COCO 저장 ===
def dump_subset(src, keep_img_ids: set, out_json: str):
    keep = set(keep_img_ids)
    images = [im for im in src["images"] if im["id"] in keep]
    ann    = [a for a in src["annotations"] if a["image_id"] in keep]
    out = {"images": images, "annotations": ann, "categories": src["categories"]}
    save_coco(out, out_json)

for fold_id, tr_ids, va_ids in fold_indices:
    fold_dir = f"{KFOLD_DIR}/fold{fold_id}"
    os.makedirs(fold_dir, exist_ok=True)
    dump_subset(d, tr_ids, f"{fold_dir}/train.json")
    dump_subset(d, va_ids, f"{fold_dir}/val.json")
    print(f"✅ fold{fold_id} saved → train:{len(tr_ids)} / val:{len(va_ids)}")

✅ fold1 saved → train:466 / val:120
✅ fold2 saved → train:468 / val:118
✅ fold3 saved → train:464 / val:122
✅ fold4 saved → train:465 / val:121
✅ fold5 saved → train:466 / val:120


## yolo 변환

In [ ]:
# === Cell 7: COCO → YOLO 변환(폴드별) ===
from pycocotools.coco import COCO

def convert_coco_to_yolo(train_json, val_json, img_dir, yolo_dir):
    os.makedirs(yolo_dir, exist_ok=True)
    for s in ["train","val"]:
        os.makedirs(f"{yolo_dir}/images/{s}", exist_ok=True)
        os.makedirs(f"{yolo_dir}/labels/{s}", exist_ok=True)

    coco_tr = COCO(train_json); coco_va = COCO(val_json)
    cat_ids = sorted(set(coco_tr.getCatIds()) | set(coco_va.getCatIds()))
    if not cat_ids:
        cat_ids = [0]
        names = ["obj"]
    else:
        names = []
        for cid in cat_ids:
            try:
                names.append(coco_tr.loadCats([cid])[0]["name"])
            except:
                names.append(str(cid))

    catid_to_idx = {cid:i for i,cid in enumerate(cat_ids)}
    with open(f"{yolo_dir}/catid_to_idx.json","w",encoding="utf-8") as f:
        json.dump(catid_to_idx, f, ensure_ascii=False, indent=2)

    with open(f"{yolo_dir}/data_abs.yaml","w",encoding="utf-8") as f:
        f.write(f"train: {os.path.abspath(yolo_dir)}/images/train\n")
        f.write(f"val:   {os.path.abspath(yolo_dir)}/images/val\n\n")
        f.write(f"nc: {len(cat_ids)}\n")
        f.write("names: " + json.dumps(names, ensure_ascii=False) + "\n")

    def _safe_find(root, filename):
        p = os.path.join(root, filename)
        if os.path.isfile(p): return p
        hits = glob.glob(os.path.join(root, "**", Path(filename).name), recursive=True)
        return hits[0] if hits else None

    def _one_split(coco_json, split):
        coco = COCO(coco_json)
        for iid in coco.getImgIds():
            im = coco.loadImgs([iid])[0]
            fn = Path(im["file_name"]).name
            W,H = im["width"], im["height"]
            src = _safe_find(img_dir, fn)
            if not src:
                print(f"[skip missing] {fn}")
                continue
            dst_img = os.path.join(yolo_dir, "images", split, fn)
            if not os.path.isfile(dst_img):
                shutil.copy2(src, dst_img)
            anns = coco.loadAnns(coco.getAnnIds(imgIds=[iid]))
            with open(os.path.join(yolo_dir, "labels", split, Path(fn).stem + ".txt"), "w") as f:
                for a in anns:
                    cid = int(a["category_id"])
                    if cid not in catid_to_idx:
                        continue
                    x,y,w,h = a["bbox"]
                    xc = (x + w/2) / W
                    yc = (y + h/2) / H
                    f.write(f"{catid_to_idx[cid]} {xc} {yc} {w/W} {h/H}\n")

    _one_split(train_json, "train")
    _one_split(val_json,   "val")
    return f"{yolo_dir}/data_abs.yaml"

yolo_data_map = {}
for fold_id, _, _ in fold_indices:
    fold_dir = f"{KFOLD_DIR}/fold{fold_id}"
    yolo_dir = f"{YOLO_ROOT}/fold{fold_id}"
    data_yaml = convert_coco_to_yolo(
        train_json=f"{fold_dir}/train.json",
        val_json=f"{fold_dir}/val.json",
        img_dir=TRAIN_IMG_DIR,
        yolo_dir=yolo_dir
    )
    yolo_data_map[fold_id] = data_yaml
    print(f"✅ fold{fold_id} YOLO ready → {data_yaml}")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✅ fold1 YOLO ready → ./work_yolov8/yolo_splits/fold1/data_abs.yaml
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✅ fold2 YOLO ready → ./work_yolov8/yolo_splits/fold2/data_abs.yaml
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
D

## Detector 학습(ULTRALYTICS YOLO) — 과적합 스위치 포함

In [ ]:
# === Cell 8: YOLO 학습(폴드 선택) ===
from ultralytics import YOLO
import torch, gc

FOLD_TO_TRAIN = 1                # 학습할 폴드 선택
BASE_MODEL    = "yolov8m.pt"     # yolov8n/s/m/l 등 선택
IMGSZ         = 640
DEVICE        = 0
NUM_WORKERS   = 2
BATCH         = 16
EPOCHS        = 150

# 과적합 스위치: 배치 줄이기 / 데이터증강 끄기 / 얼리스탑 제거
OVERFIT_MODE  = False  # True로 켜면 과적합 유도

if OVERFIT_MODE:
    BATCH  = max(4, BATCH//2)
    EPOCHS = max(EPOCHS, 200)
    AUG_KW = dict(mosaic=0.0, mixup=0.0, copy_paste=0.0, fliplr=0.0, flipud=0.0,
                  degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
                  hsv_h=0.0, hsv_s=0.0, hsv_v=0.0)
    PATIENCE = 9999   # 사실상 얼리스탑 해제
else:
    AUG_KW = dict(mosaic=0.10, mixup=0.05, copy_paste=0.0, fliplr=0.5, flipud=0.05,
                  degrees=5.0, translate=0.08, scale=0.20, shear=1.0,
                  hsv_h=0.015, hsv_s=0.45, hsv_v=0.35)
    PATIENCE = 30

data_yaml = yolo_data_map[FOLD_TO_TRAIN]

def reset_runtime():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False)

reset_runtime()
exp_name = f"det_fold{FOLD_TO_TRAIN}{'_overfit' if OVERFIT_MODE else ''}"
run_dir  = os.path.join(WORK_DIR, exp_name)
m = YOLO(BASE_MODEL)
m.train(
    data=data_yaml, imgsz=IMGSZ, epochs=EPOCHS, device=DEVICE,
    batch=BATCH, workers=NUM_WORKERS, project=WORK_DIR, name=exp_name,
    patience=PATIENCE, verbose=True, **AUG_KW
)
r = m.val(data=data_yaml, imgsz=IMGSZ, workers=NUM_WORKERS)
print("📊 mAP@[.5:.95] =", float(r.box.map))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./work_yolov8/yolo_splits/fold1/data_abs.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.05, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.45, hsv_v=0.35, imgsz=640, int8=Fals

In [ ]:
# === Cell 8 (업데이트): YOLO 학습(단일/다중/전체 폴드 선택) ===
import os, gc, glob, time, random
import numpy as np
import torch
from ultralytics import YOLO

# ── 사용자 설정 ───────────────────────────────────────────────────────────
# 1) 하나만: 1  |  2) 일부: [1,3]  |  3) 전부: "all"
FOLD_TO_TRAIN = "all"            # 1, [1,2], "all" 중 택1
BASE_MODEL    = "yolov8m.pt"     # yolov8n/s/m/l
IMGSZ         = 640
DEVICE        = 0
NUM_WORKERS   = 2
BATCH         = 16
EPOCHS        = 150
SEED          = 42

# 과적합 스위치: 배치 줄이기 / 증강 OFF / 얼리스탑 해제
OVERFIT_MODE  = False
RESUME_IF_EXISTS = True          # 동일 exp 폴더에 last.pt 있으면 이어서 학습
USE_AMP = True                   # 자동 혼합정밀(대부분 True가 이득)

# ── 준비(전역 변수 존재 가정) ───────────────────────────────────────────
assert isinstance(yolo_data_map, dict) and len(yolo_data_map) >= 2, "yolo_data_map이 필요합니다."
WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

# ── 유틸 ─────────────────────────────────────────────────────────────────
def reset_runtime():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False)

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def latest_weight(run_dir):
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

# 증강/얼리스탑
if OVERFIT_MODE:
    BATCH  = max(4, BATCH // 2)
    EPOCHS = max(EPOCHS, 200)
    AUG_KW = dict(mosaic=0.0, mixup=0.0, copy_paste=0.0, fliplr=0.0, flipud=0.0,
                  degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
                  hsv_h=0.0, hsv_s=0.0, hsv_v=0.0)
    PATIENCE = 9999
else:
    AUG_KW = dict(mosaic=0.10, mixup=0.05, copy_paste=0.0, fliplr=0.5, flipud=0.05,
                  degrees=5.0, translate=0.08, scale=0.20, shear=1.0,
                  hsv_h=0.015, hsv_s=0.45, hsv_v=0.35)
    PATIENCE = 30

# 학습 대상 폴드 목록 만들기
if isinstance(FOLD_TO_TRAIN, str) and FOLD_TO_TRAIN.lower() == "all":
    folds_to_run = sorted(yolo_data_map.keys())
elif isinstance(FOLD_TO_TRAIN, (list, tuple, set)):
    folds_to_run = sorted(int(f) for f in FOLD_TO_TRAIN)
else:
    folds_to_run = [int(FOLD_TO_TRAIN)]

# ── 학습 루프 ────────────────────────────────────────────────────────────
results = []
for fold in folds_to_run:
    assert fold in yolo_data_map, f"fold {fold}가 yolo_data_map에 없음"
    data_yaml = yolo_data_map[fold]

    set_all_seeds(SEED + fold)   # 폴드별로 다른 시드
    reset_runtime()

    exp_name = f"det_fold{fold}{'_overfit' if OVERFIT_MODE else ''}"
    run_dir  = os.path.join(WORK_DIR, exp_name)
    resume_path = latest_weight(run_dir) if RESUME_IF_EXISTS else None

    print(f"\n==================== [TRAIN FOLD {fold}] ====================")
    print("data:", data_yaml)
    print("resume:", bool(resume_path), "| amp:", USE_AMP)

    m = YOLO(resume_path if resume_path else BASE_MODEL)

    # train
    m.train(
        data=data_yaml, imgsz=IMGSZ, epochs=EPOCHS, device=DEVICE,
        batch=BATCH, workers=NUM_WORKERS, project=WORK_DIR, name=exp_name,
        patience=PATIENCE, verbose=True, amp=USE_AMP, **AUG_KW
    )

    # val
    r = m.val(data=data_yaml, imgsz=IMGSZ, workers=NUM_WORKERS)
    best = latest_weight(run_dir)
    print("📊 mAP@[.5:.95] =", float(r.box.map), "| best:", best)

    results.append({
        "fold": fold,
        "exp_name": exp_name,
        "data_yaml": data_yaml,
        "best_weight": best,
        "map@[.5:.95]": float(r.box.map),
        "map@50": float(r.box.map50),
        "map@75": float(r.box.map75),
    })

# ── 요약 저장 ────────────────────────────────────────────────────────────
import pandas as pd
cv_df = pd.DataFrame(results).sort_values("fold")
cv_csv = os.path.join(WORK_DIR, "cv_results_summary_cell8.csv")
cv_df.to_csv(cv_csv, index=False, encoding="utf-8-sig")
print("\n✅ CV Summary saved:", cv_csv)
print(cv_df)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

==================== [TRAIN FOLD 1] ====================
data: ./work_yolov8/yolo_splits/fold1/data_abs.yaml
resume: False | amp: True
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./work_yolov8/yolo_splits/fold1/data_abs.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, flipl

In [ ]:
# === Cell 8 (업데이트 v2): YOLO 학습(단일/다중/전체 폴드 + resume + rect + 옵션 홀드아웃 예측) ===
import os, gc, glob, time, random
import numpy as np
import torch
import pandas as pd
from pathlib import Path
from ultralytics import YOLO

# ── 사용자 설정 ───────────────────────────────────────────────────────────
# 1) 하나만: 1  |  2) 일부: [1,3]  |  3) 전부: "all"
FOLD_TO_TRAIN = "all"            # 1, [1,2], "all" 중 택1

BASE_MODEL    = "yolov8m.pt"     # yolov8n/s/m/l
IMGSZ         = 1024             # 976×1280 세로형에 맞춰 1280 권장(32의 배수)
DEVICE        = 0
NUM_WORKERS   = 2
BATCH         = 16
EPOCHS        = 150
SEED          = 42

# 과적합 스위치: 배치 줄이기 / 증강 OFF / 얼리스탑 해제
OVERFIT_MODE      = False
RESUME_IF_EXISTS  = True          # 동일 exp 폴더에 last/best 있으면 이어서 학습
USE_AMP           = True          # 자동 혼합정밀
USE_MULTI_SCALE   = True          # 여유 있으면 True
RECT_TRAIN_VAL    = True          # ✅ 세로 비율 보존(왜곡 방지)
EXPORT_BEST_ON_END= True          # 학습 종료 후 best.pt를 onnx로도 내보내기(옵션)

# (옵션) 홀드아웃 테스트 예측
DO_HOLDOUT_PRED   = True          # Cell 4.5에서 만든 TEST_DIR/_holdout_imgs 사용
TEST_CONF         = 0.20
TEST_IOU          = 0.65
TEST_TOPK         = 4
TEST_TTA          = True
TEST_AGNOSTIC_NMS = True

# ── 준비(전역 변수 존재 가정) ───────────────────────────────────────────
# - yolo_data_map: {fold:int -> data_yaml:str} (Cell 7에서 생성)
# - WORK_DIR: 작업 폴더
assert isinstance(yolo_data_map, dict) and len(yolo_data_map) >= 2, "yolo_data_map이 필요합니다."
WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

# Cell 4.5가 있었다면 TEST_DIR이 전역에 있을 것임(없어도 학습엔 문제 없음)
TEST_DIR = globals().get("TEST_DIR", None)
if TEST_DIR is not None:
    TEST_DIR = os.path.abspath(TEST_DIR)

# ── 유틸 ─────────────────────────────────────────────────────────────────
def reset_runtime():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False)

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def latest_weight(run_dir):
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

def predict_to_csv(model_or_path, test_dir, out_csv, imgsz=IMGSZ,
                   conf=TEST_CONF, iou=TEST_IOU, max_det=2000, topk=TEST_TOPK,
                   tta=TEST_TTA, agnostic=TEST_AGNOSTIC_NMS):
    """간단 홀드아웃 예측 저장기"""
    from ultralytics import YOLO as _YOLO
    rows, ann_id = [], 1
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    image_files = [f for f in os.listdir(test_dir)
                   if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]
    image_files.sort()
    for fn in image_files:
        ip = os.path.join(test_dir, fn)
        r = model.predict(
            source=ip, imgsz=imgsz, conf=conf, iou=iou,
            agnostic_nms=agnostic, max_det=max_det,
            augment=tta, verbose=False, rect=RECT_TRAIN_VAL
        )[0]
        if not r.boxes or len(r.boxes) == 0:
            continue
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)

        order = np.argsort(-confs)[:topk]
        for i in order:
            x1, y1, x2, y2 = boxes[i]
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": int(clss[i]),
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1

    df = pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_csv, len(df)

# 증강/얼리스탑
if OVERFIT_MODE:
    BATCH  = max(4, BATCH // 2)
    EPOCHS = max(EPOCHS, 200)
    AUG_KW = dict(
        mosaic=0.0, mixup=0.0, copy_paste=0.0, fliplr=0.0, flipud=0.0,
        degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
        hsv_h=0.0, hsv_s=0.0, hsv_v=0.0
    )
    PATIENCE = 9999
else:
    # 세로형 데이터에 무리 없는 기본 증강(모자이크/스케일 적당, flipud 낮게)
    AUG_KW = dict(
        mosaic=0.08, mixup=0.03, copy_paste=0.0, fliplr=0.5, flipud=0.02,
        degrees=3.0, translate=0.06, scale=0.20, shear=0.5,
        hsv_h=0.010, hsv_s=0.40, hsv_v=0.30,
        close_mosaic=12
    )
    PATIENCE = 30

# 학습 대상 폴드 목록 만들기
if isinstance(FOLD_TO_TRAIN, str) and FOLD_TO_TRAIN.lower() == "all":
    folds_to_run = sorted(yolo_data_map.keys())
elif isinstance(FOLD_TO_TRAIN, (list, tuple, set)):
    folds_to_run = sorted(int(f) for f in FOLD_TO_TRAIN)
else:
    folds_to_run = [int(FOLD_TO_TRAIN)]

# ── 학습 루프 ────────────────────────────────────────────────────────────
results = []
for fold in folds_to_run:
    assert fold in yolo_data_map, f"fold {fold}가 yolo_data_map에 없음"
    data_yaml = yolo_data_map[fold]

    set_all_seeds(SEED + fold)   # 폴드별로 다른 시드
    reset_runtime()

    exp_name = f"det_fold{fold}{'_overfit' if OVERFIT_MODE else ''}"
    run_dir  = os.path.join(WORK_DIR, exp_name)
    resume_path = latest_weight(run_dir) if RESUME_IF_EXISTS else None

    print(f"\n==================== [TRAIN FOLD {fold}] ====================")
    print("data:", data_yaml)
    print("resume:", bool(resume_path), "| amp:", USE_AMP, "| rect:", RECT_TRAIN_VAL, "| multi_scale:", USE_MULTI_SCALE)

    m = YOLO(resume_path if resume_path else BASE_MODEL)

    # train
    m.train(
        data=data_yaml,
        imgsz=IMGSZ,
        epochs=EPOCHS,
        device=DEVICE,
        batch=BATCH,
        workers=NUM_WORKERS,
        project=WORK_DIR,
        name=exp_name,
        patience=PATIENCE,
        verbose=True,
        amp=USE_AMP,
        rect=RECT_TRAIN_VAL,          # ✅ 비율 보존 학습
        multi_scale=USE_MULTI_SCALE,  # 메모리/속도 여유에 따라
        **AUG_KW
    )

    # val (학습과 동일 해상도/rect로 평가 일치)
    r = m.val(data=data_yaml, imgsz=IMGSZ, workers=NUM_WORKERS, rect=RECT_TRAIN_VAL)
    best = latest_weight(run_dir)
    print("📊 mAP@[.5:.95] =", float(r.box.map), "| best:", best)

    # (옵션) 내보내기
    if EXPORT_BEST_ON_END and best and os.path.isfile(best):
        try:
            m_best = YOLO(best)
            m_best.export(format="onnx", imgsz=IMGSZ, half=False, dynamic=False)  # 필요 시 수정
        except Exception as e:
            print("export skipped:", e)

    # (옵션) 홀드아웃 예측
    pred_csv = None
    if DO_HOLDOUT_PRED and TEST_DIR and os.path.isdir(TEST_DIR) and best:
        ts = int(time.time())
        pred_csv = os.path.join(WORK_DIR, f"pred_holdout_fold{fold}_{ts}.csv")
        predict_to_csv(best, TEST_DIR, pred_csv, imgsz=IMGSZ)
        print("🧪 holdout csv:", pred_csv)

    results.append({
        "fold": fold,
        "exp_name": exp_name,
        "data_yaml": data_yaml,
        "best_weight": best,
        "map@[.5:.95]": float(r.box.map),
        "map@50": float(r.box.map50),
        "map@75": float(r.box.map75),
        "holdout_csv": pred_csv
    })

# ── 요약 저장 ────────────────────────────────────────────────────────────
cv_df = pd.DataFrame(results).sort_values("fold")
cv_csv = os.path.join(WORK_DIR, "cv_results_summary_cell8_v2.csv")
cv_df.to_csv(cv_csv, index=False, encoding="utf-8-sig")
print("\n✅ CV Summary saved:", cv_csv)
print(cv_df)


==================== [TRAIN FOLD 1] ====================
data: ./work_yolov8/yolo_splits/fold1/data_abs.yaml
resume: True | amp: True | rect: True | multi_scale: True
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=12, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./work_yolov8/yolo_splits/fold1/data_abs.yaml, degrees=3.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.02, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.4, hsv_v=0.3, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.03, mode=train, model=/content/work_yo

OutOfMemoryError: CUDA out of memory. Tried to allocate 184.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 167.38 MiB is free. Process 6044 has 21.98 GiB memory in use. Of the allocated memory 20.67 GiB is allocated by PyTorch, and 1.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 추론(TTA) → CSV (catid 매핑 반영)

In [ ]:
# === Cell 9: Inference(TTA) → CSV ===
from ultralytics import YOLO as _YOLO
import time

def load_idx_to_catid(yolo_dir_for_split):
    mapping_json = os.path.join(yolo_dir_for_split, "catid_to_idx.json")
    if os.path.isfile(mapping_json):
        catid_to_idx = json.load(open(mapping_json, "r"))
        return {int(v): int(k) for k, v in catid_to_idx.items()}
    return None

def predict_to_csv(model_or_path, test_dir, out_csv, imgsz=640,
                   conf=0.25, iou=0.65, max_det=2000, topk=4,
                   tta=True, agnostic=True, idx_to_catid=None):
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    rows, ann_id = [], 1
    files = [f for f in os.listdir(test_dir) if f.lower().endswith(IMG_EXTS)]
    files.sort()
    for fn in files:
        ip = os.path.join(test_dir, fn)
        r = model.predict(source=ip, imgsz=imgsz, conf=conf, iou=iou,
                          agnostic_nms=agnostic, max_det=max_det,
                          augment=tta, verbose=False)[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)
        order = np.argsort(-confs)[:topk]
        for i in order:
            x1, y1, x2, y2 = boxes[i]
            cid = int(idx_to_catid.get(clss[i], clss[i])) if idx_to_catid else int(clss[i])
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": cid,
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1
    pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ]).to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_csv, len(rows)

yolo_dir_for_split = os.path.dirname(yolo_data_map[FOLD_TO_TRAIN])
idx_to_catid = load_idx_to_catid(yolo_dir_for_split)

# best.pt 우선
best_pt = os.path.join(WORK_DIR, f"det_fold{FOLD_TO_TRAIN}{'_overfit' if OVERFIT_MODE else ''}", "weights", "best.pt")
last_pt = os.path.join(WORK_DIR, f"det_fold{FOLD_TO_TRAIN}{'_overfit' if OVERFIT_MODE else ''}", "weights", "last.pt")
weight  = best_pt if os.path.isfile(best_pt) else last_pt
timestamp = int(time.time())
csv_out = os.path.join(PRED_DIR, f"pred_fold{FOLD_TO_TRAIN}_{'overfit_' if OVERFIT_MODE else ''}{timestamp}.csv")
csv_path, nrow = predict_to_csv(weight, TEST_IMG_DIR, csv_out, imgsz=640,
                                conf=0.20, iou=0.65, topk=4,
                                tta=True, agnostic=True, idx_to_catid=idx_to_catid)
print(f"✅ CSV saved: {csv_path} (rows={nrow})")

✅ CSV saved: ./work_yolov8/predictions/pred_fold1_1761703859.csv (rows=3228)


In [ ]:
# === Cell A: TEST_DIR 점검 및 _holdout_imgs 재구축(복사) ===
import os, json, shutil
from pathlib import Path

WORK_DIR       = os.path.abspath("./work_yolov8")
TRAIN_IMG_DIR  = os.path.abspath("./data/ai05-level1-project/train_images")
TEST_JSON      = os.path.join(WORK_DIR, "test_holdout.json")   # Cell 4.5가 만든 파일
TEST_DIR_LINK  = os.path.join(WORK_DIR, "_holdout_imgs")       # 실제 추론 경로(폴더)

# 0) 기본 점검
assert os.path.isfile(TEST_JSON), f"TEST_JSON not found: {TEST_JSON}"
assert os.path.isdir(TRAIN_IMG_DIR), f"TRAIN_IMG_DIR not found: {TRAIN_IMG_DIR}"

# 1) test 목록 로드
with open(TEST_JSON, "r", encoding="utf-8") as f:
    dtest = json.load(f)
name_set = {Path(im["file_name"]).name for im in dtest.get("images", [])}

# 2) _holdout_imgs 를 '복사' 방식으로 재구축(심볼릭 링크 X)
os.makedirs(TEST_DIR_LINK, exist_ok=True)
# 폴더 비우기
for p in Path(TEST_DIR_LINK).glob("*"):
    try:
        if p.is_symlink() or p.is_file(): p.unlink()
        elif p.is_dir(): shutil.rmtree(p)
    except Exception:
        pass

missing, copied = [], 0
for name in sorted(name_set):
    src = Path(TRAIN_IMG_DIR) / name
    dst = Path(TEST_DIR_LINK) / name
    if not src.exists():
        missing.append(name)
        continue
    shutil.copy2(src, dst)
    copied += 1

print(f"📦 Rebuilt TEST_DIR by copy → {TEST_DIR_LINK}")
print(f"   copied: {copied}  | missing in TRAIN_IMG_DIR: {len(missing)}")
if missing[:10]:
    print("   missing samples:", missing[:10])

# 3) 전역 TEST_DIR 갱신(아래 추론 셀들이 이 값을 사용)
TEST_DIR = TEST_DIR_LINK
print("✅ TEST_DIR set to:", TEST_DIR)

In [ ]:
# === Cell Z: 점검 → _holdout_imgs 복구(copy) → 안전 추론(holdout→test) ===
import os, glob, json, time, shutil
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO as _YOLO

# ---- 공용 설정/전역 (앞 셀에서 정의한 값 재사용; 없으면 기본값) ----
WORK_DIR       = globals().get("WORK_DIR", os.path.abspath("./work_yolov8"))
TRAIN_IMG_DIR  = globals().get("TRAIN_IMG_DIR", os.path.abspath("./data/ai05-level1-project/train_images"))
OFFICIAL_TEST_DIR = os.path.abspath("./data/ai05-level1-project/test_images")  # 공식 test 폴더
TEST_JSON      = globals().get("TEST_JSON", os.path.join(WORK_DIR, "test_holdout.json"))
HOLDOUT_DIR    = os.path.join(WORK_DIR, "_holdout_imgs")  # 항상 여기서 추론
PRED_DIR       = os.path.join(WORK_DIR, "pred_csv")
os.makedirs(PRED_DIR, exist_ok=True)

# 학습 결과 접근용
assert "yolo_data_map" in globals() and len(yolo_data_map) >= 1, "yolo_data_map 준비 필요"
FOLD_TO_TRAIN  = globals().get("FOLD_TO_TRAIN", "all")
IMGSZ_INFER    = globals().get("IMGSZ", 640)
CONF_THRESH    = 0.20
IOU_THRESH     = 0.65
TOPK           = 4
TTA            = True
AGNOSTIC_NMS   = True
IMG_EXTS       = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

# ---- 0) 유틸 ----
def yolo_fold_dir_from_yaml(data_yaml_path):
    return os.path.dirname(os.path.abspath(data_yaml_path))

def latest_weight(run_dir):
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

def load_idx_to_catid(yolo_fold_dir):
    p = os.path.join(yolo_fold_dir, "catid_to_idx.json")
    if os.path.isfile(p):
        catid_to_idx = json.load(open(p, "r", encoding="utf-8"))
        return {int(v): int(k) for k, v in catid_to_idx.items()}
    return None

def _safe_find(root_dirs, filename):
    """filename을 여러 루트에서 재탐색(glob)하여 실제 경로 반환"""
    for rd in (root_dirs if isinstance(root_dirs, (list, tuple)) else [root_dirs]):
        # 직행
        cand = os.path.join(rd, filename)
        if os.path.isfile(cand):
            return cand
        # 재귀 탐색
        hits = glob.glob(os.path.join(rd, "**", Path(filename).name), recursive=True)
        if hits:
            return hits[0]
    return None

# ---- 1) 진단: 현재 폴더 상태 확인 ----
def _count_images(d):
    if not os.path.isdir(d): return 0
    return sum(1 for f in os.listdir(d) if f.lower().endswith(IMG_EXTS))

print("📋 상태 점검")
print(" - TRAIN_IMG_DIR:", TRAIN_IMG_DIR, "| imgs:", _count_images(TRAIN_IMG_DIR))
print(" - OFFICIAL_TEST_DIR:", OFFICIAL_TEST_DIR, "| imgs:", _count_images(OFFICIAL_TEST_DIR))
print(" - TEST_JSON:", os.path.isfile(TEST_JSON))

# ---- 2) _holdout_imgs를 '복사'로 재구축 (심볼릭 링크 금지) ----
if os.path.isfile(TEST_JSON):
    with open(TEST_JSON, "r", encoding="utf-8") as f:
        dtest = json.load(f)
    wanted_names = {Path(im["file_name"]).name for im in dtest.get("images", [])}
else:
    # TEST_JSON 없으면 안전하게 비워둠
    wanted_names = set()

os.makedirs(HOLDOUT_DIR, exist_ok=True)
# 폴더 비우기
for p in Path(HOLDOUT_DIR).glob("*"):
    try:
        if p.is_symlink() or p.is_file(): p.unlink()
        elif p.is_dir(): shutil.rmtree(p)
    except Exception:
        pass

copied, missing = 0, []
for name in sorted(wanted_names):
    src = _safe_find(TRAIN_IMG_DIR, name)  # train 루트에서 탐색
    if not src:
        missing.append(name)
        continue
    shutil.copy2(src, os.path.join(HOLDOUT_DIR, name))
    copied += 1

print(f"🧱 _holdout_imgs 재구축 완료 → {HOLDOUT_DIR} | copied={copied}, missing={len(missing)}")
if missing[:10]:
    print("   missing 예시:", missing[:10])

# ---- 3) 안전 추론 함수 (끊어진 파일 자동 스킵 + 대체 탐색) ----
def predict_to_csv(model_or_path, img_dir, out_csv, imgsz=640,
                   conf=0.20, iou=0.65, max_det=2000, topk=4,
                   tta=True, agnostic=True, idx_to_catid=None,
                   fallback_dirs=None):
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    rows, ann_id = [], 1
    files = [f for f in os.listdir(img_dir) if f.lower().endswith(IMG_EXTS)]
    files.sort()

    total, repaired, skipped = 0, 0, 0
    for fn in files:
        total += 1
        ip = os.path.join(img_dir, fn)
        if not os.path.exists(ip) and fallback_dirs:
            alt = _safe_find(fallback_dirs, fn)
            if alt:
                ip = alt
                repaired += 1
        if not os.path.exists(ip):
            skipped += 1
            continue

        r = model.predict(
            source=ip, imgsz=imgsz, conf=conf, iou=iou,
            agnostic_nms=agnostic, max_det=max_det,
            augment=tta, verbose=False
        )[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue

        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)
        order = np.argsort(-confs)[:topk]

        for i in order:
            x1, y1, x2, y2 = boxes[i]
            cid = int(idx_to_catid.get(clss[i], clss[i])) if idx_to_catid else int(clss[i])
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": cid,
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1

    pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ]).to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"   → scanned: {total}, repaired: {repaired}, skipped: {skipped}, rows: {len(rows)}")
    return out_csv, len(rows)

# ---- 4) 폴드 루프: 1) holdout → 2) official test 순서로 추론 ----
if isinstance(FOLD_TO_TRAIN, str) and FOLD_TO_TRAIN.lower() == "all":
    folds_to_run = sorted(yolo_data_map.keys())
elif isinstance(FOLD_TO_TRAIN, (list, tuple, set)):
    folds_to_run = sorted(int(f) for f in FOLD_TO_TRAIN)
else:
    folds_to_run = [int(FOLD_TO_TRAIN)]

ts = int(time.time())

def run_for(img_dir, tag):
    fold_csvs = []
    for fold in folds_to_run:
        data_yaml    = yolo_data_map[fold]
        yolo_folddir = yolo_fold_dir_from_yaml(data_yaml)

        exp_name = f"det_fold{fold}{'_overfit' if globals().get('OVERFIT_MODE', False) else ''}"
        run_dir  = os.path.join(WORK_DIR, exp_name)
        weight   = latest_weight(run_dir)
        assert weight and os.path.isfile(weight), f"가중치 없음: {run_dir}/weights"

        idx_to_catid = load_idx_to_catid(yolo_folddir)
        out_csv = os.path.join(PRED_DIR, f"pred_{tag}_fold{fold}_{ts}.csv")
        print(f"[{tag} | fold{fold}] img_dir = {img_dir}")
        csv_path, nrow = predict_to_csv(
            weight, img_dir, out_csv, imgsz=IMGSZ_INFER,
            conf=CONF_THRESH, iou=IOU_THRESH, topk=TOPK,
            tta=TTA, agnostic=AGNOSTIC_NMS, idx_to_catid=idx_to_catid,
            fallback_dirs=[HOLDOUT_DIR, TRAIN_IMG_DIR, OFFICIAL_TEST_DIR]
        )
        print(f"✅ {tag} fold{fold} CSV saved: {csv_path} (rows={nrow})")
        fold_csvs.append(csv_path)
    return fold_csvs

print("\n=== 1) HOLDOUT 추론 ===")
hold_csvs = run_for(HOLDOUT_DIR, "holdout")

print("\n=== 2) OFFICIAL TEST 추론 ===")
if os.path.isdir(OFFICIAL_TEST_DIR) and _count_images(OFFICIAL_TEST_DIR) > 0:
    test_csvs = run_for(OFFICIAL_TEST_DIR, "test")
else:
    print(f"⚠️ OFFICIAL_TEST_DIR 비어있음 또는 없음 → {OFFICIAL_TEST_DIR}")
    test_csvs = []

# (옵션) 폴드≥2면 간단 앙상블
def simple_ensemble(csv_list, out_path):
    if len(csv_list) < 2:
        return None
    dfs = []
    for p in csv_list:
        d = pd.read_csv(p)
        d["key"] = (
            d["image_id"].astype(str) + "|" +
            d["category_id"].astype(int).astype_str() + "|" +
            d["bbox_x"].round(1).astype_str() + "|" +
            d["bbox_y"].round(1).astype_str() + "|" +
            d["bbox_w"].round(1).astype_str() + "|" +
            d["bbox_h"].round(1).astype_str()
        )
        dfs.append(d)
    merged = pd.concat(dfs, ignore_index=True)
    grp = merged.groupby("key", as_index=False).agg({
        "image_id":"first","category_id":"first",
        "bbox_x":"first","bbox_y":"first","bbox_w":"first","bbox_h":"first",
        "score":"mean"
    }).sort_values(["image_id","score"], ascending=[True, False]).reset_index(drop=True)
    grp.insert(0, "annotation_id", np.arange(1, len(grp)+1))
    grp[["annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"]].to_csv(
        out_path, index=False, encoding="utf-8-sig"
    )
    return out_path

# === Fix: 간단 앙상블 함수 교체(astype_str -> astype(str)) ===
import os, numpy as np, pandas as pd

def simple_ensemble(csv_list, out_path, round_ndigits=1, keep_topk_per_image=None):
    """
    여러 폴드 CSV를 좌표 근접(소수 round_ndigits 자리 반올림) 기준으로 그룹핑 후
    score를 평균 내어 앙상블. 필요 시 이미지별 Top-K만 남길 수 있음.
    """
    if len(csv_list) < 2:
        print("ℹ️ 앙상블 스킵: csv_list < 2")
        return None

    dfs = []
    for p in csv_list:
        d = pd.read_csv(p)
        # 안전: 타입 정규화
        d["image_id"]   = d["image_id"].astype(str)
        d["category_id"]= d["category_id"].astype(int)
        d["bbox_x"]     = d["bbox_x"].astype(float).round(round_ndigits)
        d["bbox_y"]     = d["bbox_y"].astype(float).round(round_ndigits)
        d["bbox_w"]     = d["bbox_w"].astype(float).round(round_ndigits)
        d["bbox_h"]     = d["bbox_h"].astype(float).round(round_ndigits)
        d["score"]      = d["score"].astype(float)

        # 그룹핑 키 생성 (문자열 결합)
        d["key"] = (
            d["image_id"] + "|" +
            d["category_id"].astype(str) + "|" +
            d["bbox_x"].astype(str) + "|" +
            d["bbox_y"].astype(str) + "|" +
            d["bbox_w"].astype(str) + "|" +
            d["bbox_h"].astype(str)
        )
        dfs.append(d)

    merged = pd.concat(dfs, ignore_index=True)

    # 동일 key를 평균 score로 집계
    grp = (
        merged.groupby("key", as_index=False)
        .agg({
            "image_id":"first", "category_id":"first",
            "bbox_x":"first", "bbox_y":"first", "bbox_w":"first", "bbox_h":"first",
            "score":"mean"
        })
        .sort_values(["image_id","score"], ascending=[True, False])
        .reset_index(drop=True)
    )

    # (옵션) 이미지별 Top-K 제한
    if keep_topk_per_image is not None and keep_topk_per_image > 0:
        grp["rank"] = grp.groupby("image_id")["score"].rank(method="first", ascending=False)
        grp = grp[grp["rank"] <= keep_topk_per_image].drop(columns=["rank"]).reset_index(drop=True)

    grp.insert(0, "annotation_id", np.arange(1, len(grp)+1))
    cols = ["annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"]
    grp[cols].to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"🌀 Ensemble CSV saved: {out_path}  | rows={len(grp)}")
    return out_path

  # Holdout 앙상블
if len(hold_csvs) >= 2:
    out_ens = os.path.join(PRED_DIR, f"pred_holdout_ensemble_{ts}.csv")
    simple_ensemble(hold_csvs, out_ens, round_ndigits=1, keep_topk_per_image=None)

# Test 앙상블 (예: 이미지별 Top-4만 남기고 싶다면 keep_topk_per_image=4)
if len(test_csvs) >= 2:
    out_ens = os.path.join(PRED_DIR, f"pred_test_ensemble_{ts}.csv")
    simple_ensemble(test_csvs, out_ens, round_ndigits=1, keep_topk_per_image=4)

📋 상태 점검
 - TRAIN_IMG_DIR: ./data/ai05-level1-project/train_images | imgs: 1494
 - OFFICIAL_TEST_DIR: /content/data/ai05-level1-project/test_images | imgs: 843
 - TEST_JSON: True
🧱 _holdout_imgs 재구축 완료 → /content/work_yolov8/_holdout_imgs | copied=65, missing=0

=== 1) HOLDOUT 추론 ===
[holdout | fold1] img_dir = /content/work_yolov8/_holdout_imgs
   → scanned: 65, repaired: 0, skipped: 0, rows: 245
✅ holdout fold1 CSV saved: /content/work_yolov8/pred_csv/pred_holdout_fold1_1761728980.csv (rows=245)
[holdout | fold2] img_dir = /content/work_yolov8/_holdout_imgs
   → scanned: 65, repaired: 0, skipped: 0, rows: 244
✅ holdout fold2 CSV saved: /content/work_yolov8/pred_csv/pred_holdout_fold2_1761728980.csv (rows=244)
[holdout | fold3] img_dir = /content/work_yolov8/_holdout_imgs
   → scanned: 65, repaired: 0, skipped: 0, rows: 245
✅ holdout fold3 CSV saved: /content/work_yolov8/pred_csv/pred_holdout_fold3_1761728980.csv (rows=245)
[holdout | fold4] img_dir = /content/work_yolov8/_holdout_imgs

In [ ]:
# === Cell 9: Inference (All folds + TTA + Ensemble → CSV, one-cell) ===
import os, json, glob, time, math
from pathlib import Path
import numpy as np
import pandas as pd
from ultralytics import YOLO as _YOLO

# ── 기본 경로(필요시 수정) ─────────────────────────────────────────────────
WORK_DIR    = os.path.abspath("./work_yolov8")
DATA_ROOT   = os.path.abspath("./data/ai05-level1-project")
DEFAULT_TEST_DIR = os.path.join(WORK_DIR, "_holdout_imgs")             # Cell 4.5에서 만든 홀드아웃 이미지 폴더
FALLBACK_TEST_DIR= os.path.join(DATA_ROOT, "test_images")              # 없으면 이 경로 사용
YOLO_ROOT   = os.path.join(WORK_DIR, "yolo_dataset")
PRED_DIR    = os.path.join(WORK_DIR, "predictions")
os.makedirs(PRED_DIR, exist_ok=True)

# ── 추론 하이퍼파라미터(권장값) ────────────────────────────────────────────
IMGSZ    = 1280          # 세로 이미지(976x1280) 최적화
CONF     = 0.08          # 리콜 확보(0.05~0.10 사이 검증으로 튜닝)
IOU      = 0.58          # 0.55~0.60 권장
TOPK     = 50            # 넉넉히 뽑아서 앙상블 시 정리
TTA      = True
AGNOSTIC = True
IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

# ── 어떤 폴드를 사용할지: "all" | [1,3] | 2 ──────────────────────────────
FOLD_TO_USE   = "all"
OVERFIT_MODE  = False    # 학습 때 overfit 모드였다면 True로 맞춰 동일 폴더명 탐색

# ── yolo_data_map 자동 구성(앞 셀에서 dict가 없다면) ─────────────────────
#    yolo_dataset/fold*/data_abs.yaml 스캔으로 fold -> yaml 매핑
def build_yolo_data_map(yolo_root):
    m = {}
    for p in sorted(glob.glob(os.path.join(yolo_root, "fold*", "data_abs.yaml"))):
        try:
            fold = int(Path(p).parts[-2].replace("fold",""))
            m[fold] = os.path.abspath(p)
        except Exception:
            pass
    return m

yolo_data_map = globals().get("yolo_data_map")
if not isinstance(yolo_data_map, dict) or len(yolo_data_map) == 0:
    yolo_data_map = build_yolo_data_map(YOLO_ROOT)
    assert len(yolo_data_map) > 0, f"fold별 data_abs.yaml을 찾지 못함: {YOLO_ROOT}/fold*/data_abs.yaml"

# ── 사용할 폴드 목록 정리 ─────────────────────────────────────────────────
if isinstance(FOLD_TO_USE, str) and FOLD_TO_USE.lower() == "all":
    folds = sorted(yolo_data_map.keys())
elif isinstance(FOLD_TO_USE, (list, tuple, set)):
    folds = sorted(int(f) for f in FOLD_TO_USE)
else:
    folds = [int(FOLD_TO_USE)]

# ── 테스트 디렉토리 결정(우선순위: TEST_DIR 변수 → _holdout_imgs → data/test_images) ─
TEST_DIR = globals().get("TEST_DIR", DEFAULT_TEST_DIR)
if not os.path.isdir(TEST_DIR):
    TEST_DIR = FALLBACK_TEST_DIR
assert os.path.isdir(TEST_DIR), f"TEST_DIR not found: {TEST_DIR}"

# ── 보조 함수들 ────────────────────────────────────────────────────────────
def load_idx_to_catid(yolo_dir_for_split):
    """
    fold별 yolo 디렉토리 안의 catid_to_idx.json을 역매핑(idx->cat_id)으로 변환.
    없으면 None → (idx=cat_id)로 그대로 사용.
    """
    mapping_json = os.path.join(yolo_dir_for_split, "catid_to_idx.json")
    if os.path.isfile(mapping_json):
        with open(mapping_json, "r", encoding="utf-8") as f:
            catid_to_idx = json.load(f)  # {cat_id: idx}
        return {int(v): int(k) for k, v in catid_to_idx.items()}  # idx->cat_id
    return None

def predict_to_csv(model_or_path, test_dir, out_csv, imgsz=IMGSZ,
                   conf=CONF, iou=IOU, max_det=2000, topk=TOPK,
                   tta=TTA, agnostic=AGNOSTIC, idx_to_catid=None):
    """
    단일 모델(or 경로)로 test_dir 전부 예측해서 CSV 저장.
    """
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    rows, ann_id = [], 1
    files = [f for f in os.listdir(test_dir) if f.lower().endswith(IMG_EXTS)]
    files.sort()

    for fn in files:
        ip = os.path.join(test_dir, fn)
        r = model.predict(
            source=ip, imgsz=imgsz, conf=conf, iou=iou,
            agnostic_nms=agnostic, max_det=max_det,
            augment=tta, verbose=False
        )[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue

        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)

        order = np.argsort(-confs)[:topk]
        for i in order:
            x1, y1, x2, y2 = boxes[i]
            cid = int(idx_to_catid.get(clss[i], clss[i])) if idx_to_catid else int(clss[i])
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": cid,
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1

    df = pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_csv, len(df)

def latest_weight(run_dir):
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

# ── 폴드별 추론 실행 ──────────────────────────────────────────────────────
out_paths = []
ts = int(time.time())

for fold in folds:
    # fold 디렉토리 추정
    yolo_dir_for_split = os.path.dirname(yolo_data_map[fold])      # .../yolo_dataset/fold{n}
    exp_name = f"det_fold{fold}{'_overfit' if OVERFIT_MODE else ''}"
    run_dir  = os.path.join(WORK_DIR, exp_name)

    # 가중치 찾기
    weight = latest_weight(run_dir)
    assert weight and os.path.isfile(weight), f"가중치가 없습니다: {run_dir}/weights"

    # class 매핑 로드(idx -> 원래 category_id)
    idx_to_catid = load_idx_to_catid(yolo_dir_for_split)

    # 추론 CSV 경로
    csv_out = os.path.join(PRED_DIR, f"pred_fold{fold}_{'overfit_' if OVERFIT_MODE else ''}{ts}.csv")
    csv_path, nrow = predict_to_csv(weight, TEST_DIR, csv_out, imgsz=IMGSZ,
                                    conf=CONF, iou=IOU, topk=TOPK,
                                    tta=TTA, agnostic=AGNOSTIC, idx_to_catid=idx_to_catid)
    print(f"✅ FOLD {fold} CSV saved: {csv_path} (rows={nrow})")
    out_paths.append(csv_out)

# ── 폴드 앙상블(간단 평균) ────────────────────────────────────────────────
# 동일 (image_id, category_id, bbox 좌표를 소수점 1자리 반올림) 키로 그룹핑 후 score 평균
if len(out_paths) >= 2:
    all_df = []
    for p in out_paths:
        d = pd.read_csv(p)
        # 좌표 근접 매칭 키(0.1 단위 라운딩)
        d["key"] = (
            d["image_id"].astype(str) + "|" +
            d["category_id"].astype(int).astype(str) + "|" +
            d["bbox_x"].round(1).astype(str) + "|" +
            d["bbox_y"].round(1).astype(str) + "|" +
            d["bbox_w"].round(1).astype(str) + "|" +
            d["bbox_h"].round(1).astype(str)
        )
        all_df.append(d)

    merged = pd.concat(all_df, ignore_index=True)
    ens = merged.groupby("key", as_index=False).agg({
        "image_id":"first", "category_id":"first",
        "bbox_x":"first", "bbox_y":"first", "bbox_w":"first", "bbox_h":"first",
        "score":"mean"
    }).sort_values(["image_id","score"], ascending=[True, False]).reset_index(drop=True)
    ens.insert(0, "annotation_id", np.arange(1, len(ens)+1))

    final_csv = os.path.join(PRED_DIR, f"pred_ensemble_{ts}.csv")
    ens[["annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"]] \
        .to_csv(final_csv, index=False, encoding="utf-8-sig")
    print(f"🌀 Ensemble CSV saved: {final_csv}")
else:
    print("ℹ️ 단일 폴드 예측만 존재 → 앙상블 생략")